# Get data from The New York Times

Thanks https://towardsdatascience.com/collecting-data-from-the-new-york-times-over-any-period-of-time-3e365504004

## First, get a NYT API key:


Create a free developer’s account here. Sign in after you open the link in the verification email. Make your way to the developer’s home page.


In this notebook, we set up the functions to send and process multiple queries to the Article Search API provided by The New York Times. Each query asks for articles that contain a deaf-related phrase, such as "deaf and dumb" or "hearing-impaired". Full list of phrases below.

The Article Search API returns a max of 10 results at a time. If we got 143 hits. The current offset is 0, so this means we are on the first 10 results, or the first page. If we append `&page=2` to the query, and process that response, now the offset is 10. This confirms we are on the second page. Since each page has 10 results and we have 143 hits, we will need to go through ceil(143/10) = 15 pages to get all of them. 

Contents:
- Define functions
- Set up queries
- Get data
- Postprocessing

Import dependencies.

In [4]:
import os
import dateutil
import pandas as pd
#pd.options.display.max_colwidth = 100
import numpy as np
import requests
import time
import json
from ast import literal_eval
import datetime
from dateutil.relativedelta import relativedelta

key=""


## Define functions

In [5]:
def send_request(query, page):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json'
    url = base_url + '?fq=' + query + '&api-key=' + key + '&page=' + str(page)
    response = requests.get(url).json()
    time.sleep(6)
    return response


def parse_response(response, data):
    '''Parses and returns response as pandas data frame.'''
    
    articles = response['response']['docs'] 
    for article in articles: 
        
        # id
        data['id'].append(article['_id'])
        
        # Date
        date = dateutil.parser.parse(article['pub_date']).date()
        data['date'].append(date)
        
        # Headline
        data['headline'].append(article['headline']['main']) 
        
        # Section
        if 'section_name' in article:
            data['section'].append(article['section_name'])
        else:
            data['section'].append(None)
        
        # News desk
        if 'news_desk' in article:
            data['news_desk'].append(article['news_desk'])
        else:
            data['news_desk'].append(None)
        
        # Document type
        data['doc_type'].append(article['document_type'])
        
        # Type of material
        if 'type_of_material' in article: 
            data['material_type'].append(article['type_of_material'])
        else:
            data['material_type'].append(None)
            
        # Keywords
        keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
        data['keywords'].append(keywords)
        
        # Web URL
        if 'web_url' in article:
            data['url'].append(article['web_url'])
            
        # Author
        if 'byline' in article:
            data['byline'].append(article['byline']['original'])
        else:
            data['byline'].append(None)
            
            
def send_query(query, data, date=None):
    # If the query has already been sent, don't send again
    if query + '.csv' in os.listdir('data'): 
        print('Already have data for the term "' + query + '".\n')
        return False
    
    # If date is provided, append to query string
    query_str = queries[query]
    if date:
        query_str = query_str + date
        
    print('Querying string: ' + query_str + '\n')
    
    page_num = 0
    while True:
        response = send_request(query_str, page_num)
        offset = response['response']['meta']['offset']
        hits = response['response']['meta']['hits']
        
        if offset > hits: 
            print('Done processing results.\n')
            return True
        # If we have 2,000 hits or more, we will need to break down our query into date intervals
        elif hits >= 2000: 
            print('We have over 2,000 hits.\n')
            # Send the same query again, once for each date interval
            for date in q_dates:
                send_query(query, data, date) 
            return True
            
        print('Processing results ' + str(offset) + '—' + str(min((offset + 10), hits)) + '/' + str(hits) + '...')
        parse_response(response, data)
        page_num += 1

## Set up queries

Our goal is to analyze the usage of deaf-related terms in *The New York Times*. 

The deaf-related terms: 

- "deaf and dumb"
- "deaf-mute"
- "hearing-impaired"
- "tone deaf"
- "deaf as a post"
- "stone deaf" 
- "fell on deaf ears"
- "deaf" (excluding its presence in all of the above terms) 

Some terms have varieties. For example, take "fell on deaf ears." Taking into consideration all of its varieties (that yielded hits when testing manually), the resulting query would be 

>`body:"fell on deaf ears" OR headline:"fell on deaf ears" 
OR body:"fall on deaf ears" OR headline:"fall on deaf ears" 
OR body:"falls on deaf ears" 
OR body:"fall on a deaf ear" 
OR body:"fell on a deaf ear" 
OR body:"turn a deaf ear" OR headline:"turn a deaf ear" 
OR body:"turned deaf ear" OR headline:"turned deaf ear"
OR body:"turned a deaf ear" OR headline:"turned a deaf ear"`

Construct remaining queries. I did this by checking the API manually in the browser, adding conditions one by one. 

http://api.nytimes.com/svc/search/v2/articlesearch.json?fq= + `query` + &api-key= + `key`

In [6]:
deaf_and_dumb = ['deaf and dumb',
                 'deaf dumb']

deaf_mute = ['deaf mute',
             'deaf and mute',
             'mute deaf',
             'mute and deaf']

fell_on_deaf_ears = ['fell on deaf ears', 
                     'fall on deaf ears', 
                     'falls on deaf ears', 
                     'fall on a deaf ear',
                     'falling on deaf ears',
                     'falling on a deaf ear',
                     'turn a deaf ear', 
                     'turned deaf ears', 
                     'turned a deaf ear',
                     'turning deaf ears',
                     'turning a deaf ear']

hearing_impaired = ['hearing impaired', 
                    'hearing impairment']

tone_deaf = ['tone deaf']

deaf_as_a_post = ['deaf as a post']

stone_deaf = ['stone deaf']

deaf = ['deaf']

In [8]:
phrases = {'deaf_and_dumb': deaf_and_dumb,
           'deaf_mute': deaf_mute,
           'fell_on_deaf_ears': fell_on_deaf_ears,
           'hearing_impaired': hearing_impaired,
           'tone_deaf': tone_deaf,
           'deaf_as_a_post': deaf_as_a_post,
           'stone_deaf': stone_deaf,
           'deaf': deaf}

# Save phrases to file
with open('phrases.txt', 'w') as outfile:
    json.dump(phrases, outfile)

Construct queries based on phrases.

In [9]:
queries = {phrase: (' OR ').join(['body:"' + x + '" OR headline:"' + x + '"' for x in phrases[phrase]]) for phrase in phrases.keys()}

# Edit 'deaf' query to exclude all other phrases that contain 'deaf'
queries['deaf'] = '(' + queries['deaf'] + ') AND NOT (' + queries['deaf_and_dumb'] \
+ ') AND NOT (' + queries['deaf_mute'] + ') AND NOT (' + queries['fell_on_deaf_ears'] \
+ ') AND NOT (' + queries['tone_deaf'] + ') AND NOT (' + queries['deaf_as_a_post'] \
+ ') AND NOT (' + queries['stone_deaf'] + ')'

queries

{'deaf_and_dumb': 'body:"deaf and dumb" OR headline:"deaf and dumb" OR body:"deaf dumb" OR headline:"deaf dumb"',
 'deaf_mute': 'body:"deaf mute" OR headline:"deaf mute" OR body:"deaf and mute" OR headline:"deaf and mute" OR body:"mute deaf" OR headline:"mute deaf" OR body:"mute and deaf" OR headline:"mute and deaf"',
 'fell_on_deaf_ears': 'body:"fell on deaf ears" OR headline:"fell on deaf ears" OR body:"fall on deaf ears" OR headline:"fall on deaf ears" OR body:"falls on deaf ears" OR headline:"falls on deaf ears" OR body:"fall on a deaf ear" OR headline:"fall on a deaf ear" OR body:"falling on deaf ears" OR headline:"falling on deaf ears" OR body:"falling on a deaf ear" OR headline:"falling on a deaf ear" OR body:"turn a deaf ear" OR headline:"turn a deaf ear" OR body:"turned deaf ears" OR headline:"turned deaf ears" OR body:"turned a deaf ear" OR headline:"turned a deaf ear" OR body:"turning deaf ears" OR headline:"turning deaf ears" OR body:"turning a deaf ear" OR headline:"turnin

## Get data

Date intervals for NYT search API are formatted as `&begin_date=20120101&end_date=20121231`. We need to format date intervals because if our query returns over 2,000 results, we can't get them all. We need to make our query smaller, and we can do that by narrowing the date interval. We will do 5-year intervals.

In [10]:
start, end = [datetime.datetime.strptime("1850-01-01", "%Y-%m-%d"), datetime.datetime.today()]
interval = 5 # years
dates = [(start + relativedelta(years=x)).strftime('%Y%m%d') for x in range(0, relativedelta(end, start).years + 10, interval)]
q_dates = ['&begin_date=' + start + '&end_date=' + end for start, end in zip(dates, dates[1:])]

Send these queries to the Archive API to create CSV tables that we save to `./data/`.

In [26]:
for query in list(queries.keys()):
    # Reset the global data object
    data = {'headline': [],  
            'date': [], 
            'doc_type': [],
            'material_type': [],
            'news_desk': [],
            'section': [],
            'keywords': [],
            'url': [],
            'id': [],
            'byline': []}
    
    # Send query
    got_result = send_query(query, data)
    
    # Build and save frame from data object 
    if got_result:
        data_df = pd.DataFrame(data)
        data_df['date'] = pd.to_datetime(data_df['date'])
        data_df.to_csv('data/' + query + '.csv', index=False)
        print('Saved as ' + query + '.csv.\n')

Already have data for the term "deaf_and_dumb".

Already have data for the term "deaf_mute".

Querying string: body:"fell on deaf ears" OR headline:"fell on deaf ears" OR body:"fall on deaf ears" OR headline:"fall on deaf ears" OR body:"falls on deaf ears" OR headline:"falls on deaf ears" OR body:"fall on a deaf ear" OR headline:"fall on a deaf ear" OR body:"falling on deaf ears" OR headline:"falling on deaf ears" OR body:"falling on a deaf ear" OR headline:"falling on a deaf ear" OR body:"turn a deaf ear" OR headline:"turn a deaf ear" OR body:"turned deaf ears" OR headline:"turned deaf ears" OR body:"turned a deaf ear" OR headline:"turned a deaf ear" OR body:"turning deaf ears" OR headline:"turning deaf ears" OR body:"turning a deaf ear" OR headline:"turning a deaf ear"

Processing results 0—10/1757...
Processing results 10—20/1757...
Processing results 20—30/1757...
Processing results 30—40/1757...
Processing results 40—50/1757...
Processing results 50—60/1757...
Processing results 6

We have over 2,000 hits.

Querying string: (body:"deaf" OR headline:"deaf") AND NOT (body:"deaf and dumb" OR headline:"deaf and dumb" OR body:"deaf dumb" OR headline:"deaf dumb") AND NOT (body:"deaf mute" OR headline:"deaf mute" OR body:"deaf and mute" OR headline:"deaf and mute" OR body:"mute deaf" OR headline:"mute deaf" OR body:"mute and deaf" OR headline:"mute and deaf") AND NOT (body:"fell on deaf ears" OR headline:"fell on deaf ears" OR body:"fall on deaf ears" OR headline:"fall on deaf ears" OR body:"falls on deaf ears" OR headline:"falls on deaf ears" OR body:"fall on a deaf ear" OR headline:"fall on a deaf ear" OR body:"falling on deaf ears" OR headline:"falling on deaf ears" OR body:"falling on a deaf ear" OR headline:"falling on a deaf ear" OR body:"turn a deaf ear" OR headline:"turn a deaf ear" OR body:"turned deaf ears" OR headline:"turned deaf ears" OR body:"turned a deaf ear" OR headline:"turned a deaf ear" OR body:"turning deaf ears" OR headline:"turning deaf ears" OR b

Processing results 0—10/10...
Processing results 10—10/10...
Done processing results.

Querying string: (body:"deaf" OR headline:"deaf") AND NOT (body:"deaf and dumb" OR headline:"deaf and dumb" OR body:"deaf dumb" OR headline:"deaf dumb") AND NOT (body:"deaf mute" OR headline:"deaf mute" OR body:"deaf and mute" OR headline:"deaf and mute" OR body:"mute deaf" OR headline:"mute deaf" OR body:"mute and deaf" OR headline:"mute and deaf") AND NOT (body:"fell on deaf ears" OR headline:"fell on deaf ears" OR body:"fall on deaf ears" OR headline:"fall on deaf ears" OR body:"falls on deaf ears" OR headline:"falls on deaf ears" OR body:"fall on a deaf ear" OR headline:"fall on a deaf ear" OR body:"falling on deaf ears" OR headline:"falling on deaf ears" OR body:"falling on a deaf ear" OR headline:"falling on a deaf ear" OR body:"turn a deaf ear" OR headline:"turn a deaf ear" OR body:"turned deaf ears" OR headline:"turned deaf ears" OR body:"turned a deaf ear" OR headline:"turned a deaf ear" OR 

Processing results 0—10/48...
Processing results 10—20/48...
Processing results 20—30/48...
Processing results 30—40/48...
Processing results 40—48/48...
Done processing results.

Querying string: (body:"deaf" OR headline:"deaf") AND NOT (body:"deaf and dumb" OR headline:"deaf and dumb" OR body:"deaf dumb" OR headline:"deaf dumb") AND NOT (body:"deaf mute" OR headline:"deaf mute" OR body:"deaf and mute" OR headline:"deaf and mute" OR body:"mute deaf" OR headline:"mute deaf" OR body:"mute and deaf" OR headline:"mute and deaf") AND NOT (body:"fell on deaf ears" OR headline:"fell on deaf ears" OR body:"fall on deaf ears" OR headline:"fall on deaf ears" OR body:"falls on deaf ears" OR headline:"falls on deaf ears" OR body:"fall on a deaf ear" OR headline:"fall on a deaf ear" OR body:"falling on deaf ears" OR headline:"falling on deaf ears" OR body:"falling on a deaf ear" OR headline:"falling on a deaf ear" OR body:"turn a deaf ear" OR headline:"turn a deaf ear" OR body:"turned deaf ears" O

Processing results 90—100/120...
Processing results 100—110/120...
Processing results 110—120/120...
Processing results 120—120/120...
Done processing results.

Querying string: (body:"deaf" OR headline:"deaf") AND NOT (body:"deaf and dumb" OR headline:"deaf and dumb" OR body:"deaf dumb" OR headline:"deaf dumb") AND NOT (body:"deaf mute" OR headline:"deaf mute" OR body:"deaf and mute" OR headline:"deaf and mute" OR body:"mute deaf" OR headline:"mute deaf" OR body:"mute and deaf" OR headline:"mute and deaf") AND NOT (body:"fell on deaf ears" OR headline:"fell on deaf ears" OR body:"fall on deaf ears" OR headline:"fall on deaf ears" OR body:"falls on deaf ears" OR headline:"falls on deaf ears" OR body:"fall on a deaf ear" OR headline:"fall on a deaf ear" OR body:"falling on deaf ears" OR headline:"falling on deaf ears" OR body:"falling on a deaf ear" OR headline:"falling on a deaf ear" OR body:"turn a deaf ear" OR headline:"turn a deaf ear" OR body:"turned deaf ears" OR headline:"turned 

...

Processing results 780—790/1031...
Processing results 790—800/1031...
Processing results 800—810/1031...
Processing results 810—820/1031...
Processing results 820—830/1031...
Processing results 830—840/1031...
Processing results 840—850/1031...
Processing results 850—860/1031...
Processing results 860—870/1031...
Processing results 870—880/1031...
Processing results 880—890/1031...
Processing results 890—900/1031...
Processing results 900—910/1031...
Processing results 910—920/1031...
Processing results 920—930/1031...
Processing results 930—940/1031...
Processing results 940—950/1031...
Processing results 950—960/1031...
Processing results 960—970/1031...
Processing results 970—980/1031...
Processing results 980—990/1031...
Processing results 990—1000/1031...
Processing results 1000—1010/1031...
Processing results 1010—1020/1031...
Processing results 1020—1030/1031...
Processing results 1030—1031/1031...
Done processing results.

Querying string: (body:"deaf" OR headline:"deaf") AND N

Processing results 670—680/976...
Processing results 680—690/976...
Processing results 690—700/976...
Processing results 700—710/976...
Processing results 710—720/976...
Processing results 720—730/976...
Processing results 730—740/976...
Processing results 740—750/976...
Processing results 750—760/976...
Processing results 760—770/976...
Processing results 770—780/976...
Processing results 780—790/976...
Processing results 790—800/976...
Processing results 800—810/976...
Processing results 810—820/976...
Processing results 820—830/976...
Processing results 830—840/976...
Processing results 840—850/976...
Processing results 850—860/976...
Processing results 860—870/976...
Processing results 870—880/976...
Processing results 880—890/976...
Processing results 890—900/976...
Processing results 900—910/976...
Processing results 910—920/976...
Processing results 920—930/976...
Processing results 930—940/976...
Processing results 940—950/976...
Processing results 950—960/976...
Processing res

Processing results 430—440/1027...
Processing results 440—450/1027...
Processing results 450—460/1027...
Processing results 460—470/1027...
Processing results 470—480/1027...
Processing results 480—490/1027...
Processing results 490—500/1027...
Processing results 500—510/1027...
Processing results 510—520/1027...
Processing results 520—530/1027...
Processing results 530—540/1027...
Processing results 540—550/1027...
Processing results 550—560/1027...
Processing results 560—570/1027...
Processing results 570—580/1027...
Processing results 580—590/1027...
Processing results 590—600/1027...
Processing results 600—610/1027...
Processing results 610—620/1027...
Processing results 620—630/1027...
Processing results 630—640/1027...
Processing results 640—650/1027...
Processing results 650—660/1027...
Processing results 660—670/1027...
Processing results 670—680/1027...
Processing results 680—690/1027...
Processing results 690—700/1027...
Processing results 700—710/1027...
Processing results 7

Processing results 0—10/1243...
Processing results 10—20/1243...
Processing results 20—30/1243...
Processing results 30—40/1243...
Processing results 40—50/1243...
Processing results 50—60/1243...
Processing results 60—70/1243...
Processing results 70—80/1243...
Processing results 80—90/1243...
Processing results 90—100/1243...
Processing results 100—110/1243...
Processing results 110—120/1243...
Processing results 120—130/1243...
Processing results 130—140/1243...
Processing results 140—150/1243...
Processing results 150—160/1243...
Processing results 160—170/1243...
Processing results 170—180/1243...
Processing results 180—190/1243...
Processing results 190—200/1243...
Processing results 200—210/1243...
Processing results 210—220/1243...
Processing results 220—230/1243...
Processing results 230—240/1243...
Processing results 240—250/1243...
Processing results 250—260/1243...
Processing results 260—270/1243...
Processing results 270—280/1243...
Processing results 280—290/1243...
Proce

Processing results 750—760/891...
Processing results 760—770/891...
Processing results 770—780/891...
Processing results 780—790/891...
Processing results 790—800/891...
Processing results 800—810/891...
Processing results 810—820/891...
Processing results 820—830/891...
Processing results 830—840/891...
Processing results 840—850/891...
Processing results 850—860/891...
Processing results 860—870/891...
Processing results 870—880/891...
Processing results 880—890/891...
Processing results 890—891/891...
Done processing results.

Querying string: (body:"deaf" OR headline:"deaf") AND NOT (body:"deaf and dumb" OR headline:"deaf and dumb" OR body:"deaf dumb" OR headline:"deaf dumb") AND NOT (body:"deaf mute" OR headline:"deaf mute" OR body:"deaf and mute" OR headline:"deaf and mute" OR body:"mute deaf" OR headline:"mute deaf" OR body:"mute and deaf" OR headline:"mute and deaf") AND NOT (body:"fell on deaf ears" OR headline:"fell on deaf ears" OR body:"fall on deaf ears" OR headline:"fall 

Merge all dataframes into one, with True or False for whether they contain a particular phrase. This means we're setting these additional `bool` columns:

- deaf_and_dumb
- deaf_mute
- fall_on_deaf_ears
- hearing_impaired
- tone_deaf
- deaf_as_a_post
- stone_deaf
- deaf

In [27]:
final_df = pd.DataFrame(columns=data.keys())

# For each key
for key in queries.keys():
    # Read in its CSV
    df = pd.read_csv('data/' + key + '.csv')
    
    # Drop any duplicates for this term alone
    num_dupes = len(df[df['id'].duplicated()])
    if num_dupes > 0:
        print('Dropping duplicates: ' + str(num_dupes))
        df = df.drop_duplicates(subset='id')
    print('added to final_df the df for key ' + key + ' with ' + str(len(df)) + ' values')
    
    df[key] = True # Label each row of this df as belonging to that df
    final_df = pd.concat([final_df, df], axis=0) # Add the df to the final df
    final_df.reset_index(drop=True, inplace=True)

print('\nTotal values: ' + str(len(final_df)))

added to final_df the df for key deaf_and_dumb with 727 values
added to final_df the df for key deaf_mute with 1120 values
added to final_df the df for key fell_on_deaf_ears with 1757 values
added to final_df the df for key hearing_impaired with 1302 values
added to final_df the df for key tone_deaf with 1739 values
added to final_df the df for key deaf_as_a_post with 22 values
added to final_df the df for key stone_deaf with 65 values
Dropping duplicates: 8
added to final_df the df for key deaf with 11331 values

Total values: 18063


In [28]:
final_df

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fell_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
0,THE DEAF AND DUMB WAITER.,1885-12-03,article,Archives,None,Archives,[],https://www.nytimes.com/1885/12/03/archives/th...,nyt://article/0074c23c-1ff6-5bc7-85d9-e56a5af3...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chad Threatens to Expel Sudanese Refugees,2006-04-14,article,News,International,World,[],https://www.nytimes.com/2006/04/14/world/chad-...,nyt://article/00bb19d7-2ba6-5072-8e6b-3159730d...,By Marc Lacey,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WELFARE HOTEL CHILDREN: TOMORROW'S POOR,1987-07-16,article,News,Metropolitan Desk,New York,"['Homeless Persons', 'HOTELS AND MOTELS', 'Chi...",https://www.nytimes.com/1987/07/16/nyregion/we...,nyt://article/01670df3-ae07-5eb6-8862-7bd834bf...,By Lydia Chavez,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Wal-Mart Says Oil Prices Held Down Profits for...,2005-08-16,article,News,Business,Business Day,['Company Reports'],https://www.nytimes.com/2005/08/16/business/wa...,nyt://article/0175ac61-cc62-5cdc-923c-f5efb8ec...,By Roben Farzad,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"A Space Force? The Idea May Have Merit, Some Say",2018-06-23,article,News,Washington,U.S.,"['Space and Astronomy', 'United States Defense...",https://www.nytimes.com/2018/06/23/us/politics...,nyt://article/01b8b8a5-7d0c-592a-a283-a9ccd3d8...,By Helene Cooper,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18058,"Alexa, Awake",2020-01-03,multimedia,Interactive Feature,Opinion,Opinion,"['Privacy', 'Science and Technology']",https://www.nytimes.com/interactive/2020/01/03...,nyt://interactive/c104e18e-f797-5490-b03c-259b...,By Brian Turner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
18059,Confessions of a Dating Profile,2020-01-03,multimedia,Interactive Feature,Opinion,Opinion,"['Artificial Intelligence', 'Dating and Relati...",https://www.nytimes.com/interactive/2020/01/03...,nyt://interactive/ccaba5b4-672d-568b-9daa-b6bb...,By Eric Kaplan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
18060,How to Make Billions in E-Sports,2020-02-19,multimedia,Interactive Feature,Magazine,Magazine,"['E-Sports', 'Computer and Video Games', 'Fash...",https://www.nytimes.com/interactive/2020/02/18...,nyt://interactive/dd0dfee2-c9c6-5f58-acde-0801...,By Robert Capps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
18061,Parent-Teacher Association,2020-01-03,multimedia,Interactive Feature,Opinion,Opinion,"['Privacy', 'Education (K-12)', 'Children and ...",https://www.nytimes.com/interactive/2020/01/03...,nyt://interactive/df6cd908-52b0-5851-ac81-e7b8...,By Jessica Powell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


## Postprocessing

Go back through each key's column and set `NaN` to `False`. 

In [29]:
for key in queries.keys():
    final_df[key].fillna(False, inplace=True)
final_df

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fell_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
0,THE DEAF AND DUMB WAITER.,1885-12-03,article,Archives,None,Archives,[],https://www.nytimes.com/1885/12/03/archives/th...,nyt://article/0074c23c-1ff6-5bc7-85d9-e56a5af3...,NaN,True,False,False,False,False,False,False,False
1,Chad Threatens to Expel Sudanese Refugees,2006-04-14,article,News,International,World,[],https://www.nytimes.com/2006/04/14/world/chad-...,nyt://article/00bb19d7-2ba6-5072-8e6b-3159730d...,By Marc Lacey,True,False,False,False,False,False,False,False
2,WELFARE HOTEL CHILDREN: TOMORROW'S POOR,1987-07-16,article,News,Metropolitan Desk,New York,"['Homeless Persons', 'HOTELS AND MOTELS', 'Chi...",https://www.nytimes.com/1987/07/16/nyregion/we...,nyt://article/01670df3-ae07-5eb6-8862-7bd834bf...,By Lydia Chavez,True,False,False,False,False,False,False,False
3,Wal-Mart Says Oil Prices Held Down Profits for...,2005-08-16,article,News,Business,Business Day,['Company Reports'],https://www.nytimes.com/2005/08/16/business/wa...,nyt://article/0175ac61-cc62-5cdc-923c-f5efb8ec...,By Roben Farzad,True,False,False,False,False,False,False,False
4,"A Space Force? The Idea May Have Merit, Some Say",2018-06-23,article,News,Washington,U.S.,"['Space and Astronomy', 'United States Defense...",https://www.nytimes.com/2018/06/23/us/politics...,nyt://article/01b8b8a5-7d0c-592a-a283-a9ccd3d8...,By Helene Cooper,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18058,"Alexa, Awake",2020-01-03,multimedia,Interactive Feature,Opinion,Opinion,"['Privacy', 'Science and Technology']",https://www.nytimes.com/interactive/2020/01/03...,nyt://interactive/c104e18e-f797-5490-b03c-259b...,By Brian Turner,False,False,False,False,False,False,False,True
18059,Confessions of a Dating Profile,2020-01-03,multimedia,Interactive Feature,Opinion,Opinion,"['Artificial Intelligence', 'Dating and Relati...",https://www.nytimes.com/interactive/2020/01/03...,nyt://interactive/ccaba5b4-672d-568b-9daa-b6bb...,By Eric Kaplan,False,False,False,False,False,False,False,True
18060,How to Make Billions in E-Sports,2020-02-19,multimedia,Interactive Feature,Magazine,Magazine,"['E-Sports', 'Computer and Video Games', 'Fash...",https://www.nytimes.com/interactive/2020/02/18...,nyt://interactive/dd0dfee2-c9c6-5f58-acde-0801...,By Robert Capps,False,False,False,False,False,False,False,True
18061,Parent-Teacher Association,2020-01-03,multimedia,Interactive Feature,Opinion,Opinion,"['Privacy', 'Education (K-12)', 'Children and ...",https://www.nytimes.com/interactive/2020/01/03...,nyt://interactive/df6cd908-52b0-5851-ac81-e7b8...,By Jessica Powell,False,False,False,False,False,False,False,True


Some articles contain multiple phrases. For example an article may contain "deaf-mute" as well as "hearing-impaired." They currently show up as two separate rows, with duplicate values for everything except the deaf-mute and hearing-impaired columns. One row shows True in the deaf-mute column, and the other row shows True in the hearing-impaired column. We want to merge these two rows into one row where both columns show True.

In [30]:
if final_df['id'].duplicated().any():
    print('Duplicates: ' + str(final_df['id'].duplicated().sum()))

Duplicates: 587


I tried to process the duplicates in the data frame like this:

In [31]:
final_df.groupby(df.columns[0:10].tolist(), as_index=False).max() # DOESNT WORK, DROPS TOO MANY ROWS

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fell_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
0,'A Little Bit of Help',1985-09-07,article,News,Metropolitan Desk,New York,"['Culture', 'Awards, Decorations and Honors', ...",https://www.nytimes.com/1985/09/07/nyregion/ne...,nyt://article/d9a9b191-341c-5ca2-975e-7c26d0c3...,By Susan Heller Anderson and David W. Dunlap,False,False,False,False,False,False,False,True
1,"'Beach House,' romantic comedy at Circle Rep.",1985-03-29,article,News,Weekend Desk,Theater,['Theater'],https://www.nytimes.com/1985/03/29/theater/bro...,nyt://article/1d7824c8-8ec3-5af2-809b-f11b56b3...,By Enid Nemy,False,False,False,True,False,False,False,True
2,"'Noises Off,' With Flawless Timing",1990-07-08,article,News,Westchester Weekly Desk,New York,"['Reviews', 'Theater']",https://www.nytimes.com/1990/07/08/nyregion/th...,nyt://article/6f7f506a-308a-5679-bc58-79cfa26f...,By Alvin Klein,False,False,False,False,False,False,False,True
3,10TH ANNIVERSARY FOR 'SUMMER EVENINGS',1985-06-09,article,Review,Westchester Weekly Desk,New York,['Music'],https://www.nytimes.com/1985/06/09/nyregion/mu...,nyt://article/801429bd-4a71-5285-8a61-bf2ba672...,By Robert Sherman,False,False,False,False,False,False,False,True
4,A 'STREAMLINED' EDITION OF 1985 OSCAR AWARDS,1985-03-27,article,Review,Cultural Desk,Movies,"['MOTION PICTURES', 'Television']",https://www.nytimes.com/1985/03/27/movies/tv-r...,nyt://article/a332e0ad-1165-5bf7-891c-1f1670ea...,By Janet Maslin,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11952,‘Wonder Woman’ Could Be the Superhero Women in...,2017-06-04,article,News,Business,Business Day,"['Movies', 'Women and Girls', 'Sexual Harassme...",https://www.nytimes.com/2017/06/04/business/me...,nyt://article/234b9c6e-59ed-5dfb-a561-2939c35c...,By Jim Rutenberg,False,False,False,True,False,False,False,False
11953,‘World Wide Mind’,2011-02-15,article,Text,Science,Science,"['Books and Literature', 'Science and Technolo...",https://www.nytimes.com/2011/02/15/science/15s...,nyt://article/9c18470b-2b77-55bf-863a-fb55bef5...,By Michael Chorost,False,False,False,False,False,False,False,True
11954,‘Write When You Get Work’ Review: Backstreet B...,2018-11-22,article,Review,Weekend,Movies,['Movies'],https://www.nytimes.com/2018/11/22/movies/writ...,nyt://article/db879a75-4d65-56ff-b5ad-93ecf008...,By Jeannette Catsoulis,False,False,False,False,True,False,False,False
11955,"‘Yellow Vests’ Riot in Paris, but Their Anger ...",2018-12-02,article,News,Foreign,World,"['Yellow Vests Movement', 'Demonstrations, Pro...",https://www.nytimes.com/2018/12/02/world/europ...,nyt://article/4f02b33e-94f3-5d46-9fa9-71e3c50a...,By Adam Nossiter,False,False,False,False,False,False,False,True


But it didn't work. So I copied duplicates into a separate dataframe, processed them there, removed all copies of them from final_df, then appended the processed dupes to final_df and resorted. *shrug*

In [33]:
# Process dupes separately
dupes = pd.concat(g for _, g in final_df.groupby('id') if len(g) > 1)
dupes

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fell_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
3607,"M. McClelland, 53; Helped Experiments In Theat...",1993-09-22,article,Obituary; Biography,Obituary,Obituaries,"['Biographical Information', 'DEATHS']",https://www.nytimes.com/1993/09/22/obituaries/...,nyt://article/00742127-b642-510b-a474-a05476c5...,By Kathleen Teltsch,False,False,False,True,False,False,False,False
11882,"M. McClelland, 53; Helped Experiments In Theat...",1993-09-22,article,Obituary; Biography,Obituary,Obituaries,"['Biographical Information', 'DEATHS']",https://www.nytimes.com/1993/09/22/obituaries/...,nyt://article/00742127-b642-510b-a474-a05476c5...,By Kathleen Teltsch,False,False,False,False,False,False,False,True
3608,"THE HARD-OF-HEARING LIKE THE THEATER, TOO",1983-08-13,article,Letter,Editorial Desk,Opinion,['TERMS NOT AVAILABLE'],https://www.nytimes.com/1983/08/13/opinion/l-t...,nyt://article/007e25b1-b68b-5bc6-ada2-350415d9...,NaN,False,False,False,True,False,False,False,False
10169,"THE HARD-OF-HEARING LIKE THE THEATER, TOO",1983-08-13,article,Letter,Editorial Desk,Opinion,['TERMS NOT AVAILABLE'],https://www.nytimes.com/1983/08/13/opinion/l-t...,nyt://article/007e25b1-b68b-5bc6-ada2-350415d9...,NaN,False,False,False,False,False,False,False,True
3610,About Education,1979-03-20,article,Archives,None,Archives,"['Colleges and Universities', 'SPECIAL EDUCATI...",https://www.nytimes.com/1979/03/20/archives/ab...,nyt://article/00a73aaf-0717-5760-b9b0-e4c454bc...,By Fred M. Hechinger,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17899,Fractured Lands: How the Arab World Came Apart,2016-08-11,multimedia,Interactive Feature,Magazine,Magazine,['Middle East and North Africa Unrest (2010- )...,https://www.nytimes.com/interactive/2016/08/11...,nyt://interactive/719faca4-5757-5111-a7e4-19f1...,By SCOTT ANDERSON,False,False,False,False,False,False,False,True
4904,Deaf Club,2016-04-29,multimedia,Slideshow,T Magazine,T Magazine,"['Deafness', 'Music']",https://www.nytimes.com/slideshow/2016/04/29/t...,nyt://slideshow/3412e8a3-2e79-5269-a752-7573ec...,NaN,False,False,False,True,False,False,False,False
17934,Deaf Club,2016-04-29,multimedia,Slideshow,T Magazine,T Magazine,"['Deafness', 'Music']",https://www.nytimes.com/slideshow/2016/04/29/t...,nyt://slideshow/3412e8a3-2e79-5269-a752-7573ec...,NaN,False,False,False,False,False,False,False,True
4905,‘Magic to Do’,2011-07-25,multimedia,Slideshow,Theater,Theater,"['Theater', 'Deafness']",https://www.nytimes.com/slideshow/2009/02/12/t...,nyt://slideshow/bcb62c85-d683-5d1b-b548-0a7b9b...,NaN,False,False,False,True,False,False,False,False


Merge rows so that there is only one row for each unique id, retaining all True values. Note again that for each unique id, the only thing that differs are the columns containing the True/False values, hence us doing groupby on all the other columns so that we can apply .max() to those True/False columns.

.max() works cuz True > False

In [35]:
dupes.reset_index(drop=True, inplace=True)
dupes_fixed = dupes.groupby(dupes.columns[0:10].tolist(), as_index=False).max()
dupes_fixed

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fell_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
0,"'Beach House,' romantic comedy at Circle Rep.",1985-03-29,article,News,Weekend Desk,Theater,['Theater'],https://www.nytimes.com/1985/03/29/theater/bro...,nyt://article/1d7824c8-8ec3-5af2-809b-f11b56b3...,By Enid Nemy,False,False,False,True,False,False,False,True
1,DRUNK AGAIN,1985-03-31,article,News,Magazine Desk,Magazine,['English Language'],https://www.nytimes.com/1985/03/31/magazine/on...,nyt://article/89c35a34-5440-5541-afbf-3f8f652d...,By William Safire,False,False,False,True,False,False,False,True
2,HOPE IS OFFERED TO THE HARD-OF-HEARING,1985-08-18,article,News,Westchester Weekly Desk,New York,['TERMS NOT AVAILABLE'],https://www.nytimes.com/1985/08/18/nyregion/we...,nyt://article/82d141c9-c2b9-5324-bc57-1ad2021f...,By Rosalyn Fein,False,False,False,True,False,False,False,True
3,LISTENERS PAY A HIGH PRICE FOR LOUD MUSIC,1990-03-18,article,News,Arts and Leisure Desk,Arts,"['Music', 'Deafness']",https://www.nytimes.com/1990/03/18/arts/sound-...,nyt://article/a4d01869-7cfa-5f26-9a45-019ca8aa...,By Hans Fantel,False,False,False,True,False,False,False,True
4,Personal Health,1990-09-20,article,News,National Desk,U.S.,"['Deafness', 'Children and Youth', 'Implants',...",https://www.nytimes.com/1990/09/20/us/health-p...,nyt://article/a8eb2607-1042-5b45-92bc-85892bc0...,By Jane E. Brody,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,Your Money; Claiming a Pet As a Deduction,1981-03-28,article,News,Financial Desk,Business Day,"['ANIMALS', 'Taxation', 'Income Tax', 'Handica...",https://www.nytimes.com/1981/03/28/business/yo...,nyt://article/8aa2aceb-e543-5691-b4cc-572cfada...,By Elizabeth M. Fowler,False,False,False,True,False,False,False,True
451,"Your Typical Crowded, Swinging, Silent Bar Scene",1994-10-30,article,News,The City Weekly Desk,New York,"['Deafness', 'Bars']",https://www.nytimes.com/1994/10/30/nyregion/ne...,nyt://article/1aab92b9-5b05-50b7-bf7a-2b8cef19...,By Jennifer Kingson Bloom,False,False,False,True,False,False,False,True
452,"‘Fargo’ Recap: Dead Dogs, Spiders and Pestilence",2014-04-30,article,News,Culture,Arts,[],https://artsbeat.blogs.nytimes.com/2014/04/29/...,nyt://article/3a6161c6-023a-5968-a28e-0ea2ecb6...,By Kate Phillips,False,False,False,True,False,False,False,True
453,‘Singing’ With Their Hands,2012-02-11,article,News,Styles,Fashion & Style,"['Video Recordings and Downloads', 'Music', 'S...",https://www.nytimes.com/2012/02/12/fashion/sin...,nyt://article/0918d106-bd33-59fe-a100-cd3f9a23...,By Austin Considine,False,False,False,True,False,False,False,True


Remove all instances of duplicates from final_df.

In [36]:
final_df = final_df[~final_df['id'].isin(dupes['id'])]
final_final_lol_mybad_df = pd.concat((final_df, dupes_fixed))
final_final_lol_mybad_df.reset_index(inplace=True, drop=True)
final_final_lol_mybad_df

,headline,date,doc_type,material_type,news_desk,section,keywords,url,id,byline,deaf_and_dumb,deaf_mute,fell_on_deaf_ears,hearing_impaired,tone_deaf,deaf_as_a_post,stone_deaf,deaf
0,THE DEAF AND DUMB WAITER.,1885-12-03,article,Archives,None,Archives,[],https://www.nytimes.com/1885/12/03/archives/th...,nyt://article/0074c23c-1ff6-5bc7-85d9-e56a5af3...,NaN,True,False,False,False,False,False,False,False
1,Chad Threatens to Expel Sudanese Refugees,2006-04-14,article,News,International,World,[],https://www.nytimes.com/2006/04/14/world/chad-...,nyt://article/00bb19d7-2ba6-5072-8e6b-3159730d...,By Marc Lacey,True,False,False,False,False,False,False,False
2,WELFARE HOTEL CHILDREN: TOMORROW'S POOR,1987-07-16,article,News,Metropolitan Desk,New York,"['Homeless Persons', 'HOTELS AND MOTELS', 'Chi...",https://www.nytimes.com/1987/07/16/nyregion/we...,nyt://article/01670df3-ae07-5eb6-8862-7bd834bf...,By Lydia Chavez,True,False,False,False,False,False,False,False
3,Wal-Mart Says Oil Prices Held Down Profits for...,2005-08-16,article,News,Business,Business Day,['Company Reports'],https://www.nytimes.com/2005/08/16/business/wa...,nyt://article/0175ac61-cc62-5cdc-923c-f5efb8ec...,By Roben Farzad,True,False,False,False,False,False,False,False
4,"A Space Force? The Idea May Have Merit, Some Say",2018-06-23,article,News,Washington,U.S.,"['Space and Astronomy', 'United States Defense...",https://www.nytimes.com/2018/06/23/us/politics...,nyt://article/01b8b8a5-7d0c-592a-a283-a9ccd3d8...,By Helene Cooper,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17340,Your Money; Claiming a Pet As a Deduction,1981-03-28,article,News,Financial Desk,Business Day,"['ANIMALS', 'Taxation', 'Income Tax', 'Handica...",https://www.nytimes.com/1981/03/28/business/yo...,nyt://article/8aa2aceb-e543-5691-b4cc-572cfada...,By Elizabeth M. Fowler,False,False,False,True,False,False,False,True
17341,"Your Typical Crowded, Swinging, Silent Bar Scene",1994-10-30,article,News,The City Weekly Desk,New York,"['Deafness', 'Bars']",https://www.nytimes.com/1994/10/30/nyregion/ne...,nyt://article/1aab92b9-5b05-50b7-bf7a-2b8cef19...,By Jennifer Kingson Bloom,False,False,False,True,False,False,False,True
17342,"‘Fargo’ Recap: Dead Dogs, Spiders and Pestilence",2014-04-30,article,News,Culture,Arts,[],https://artsbeat.blogs.nytimes.com/2014/04/29/...,nyt://article/3a6161c6-023a-5968-a28e-0ea2ecb6...,By Kate Phillips,False,False,False,True,False,False,False,True
17343,‘Singing’ With Their Hands,2012-02-11,article,News,Styles,Fashion & Style,"['Video Recordings and Downloads', 'Music', 'S...",https://www.nytimes.com/2012/02/12/fashion/sin...,nyt://article/0918d106-bd33-59fe-a100-cd3f9a23...,By Austin Considine,False,False,False,True,False,False,False,True


In [37]:
print('Duplicates: ' + str(final_final_lol_mybad_df['id'].duplicated().sum()))

Duplicates: 0


In [38]:
final_final_lol_mybad_df.to_csv('data/all.csv', index=False)